属性查询的优先级
1. getattribute()， 无条件调用
2. 数据描述符：由1. 触发调用 （若人为的重载了该 getattribute() 方法，可能会导致无法调用描述符）
3. 实例对象的字典（若与描述符对象同名，会被覆盖哦）
4. 类的字典
5. 非数据描述符
6. 父类的字典
7. getattr() 方法

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4, compact=False).pprint
import attrs
from attrs import define, field

In [35]:
import pprint
pp = pprint.PrettyPrinter(indent=4, compact=False).pprint
from attrs import define, field

class Test(object):
    cls_val = 1
    def __init__(self):
        self.ins_val = 10

t = Test()
# pp.pprint(t.__dict__)
# pp.pprint(Test.__dict__)
pp(t.__dict__)
pp(Test.__dict__)

{'ins_val': 10}
mappingproxy({   '__dict__': <attribute '__dict__' of 'Test' objects>,
                 '__doc__': None,
                 '__init__': <function Test.__init__ at 0x000001EE17DF7F70>,
                 '__module__': '__main__',
                 '__weakref__': <attribute '__weakref__' of 'Test' objects>,
                 'cls_val': 1})


In [53]:
class Student:
    def __init__(self, name, math, chinese, english):
        self.name = name
        self.math = math
        self.chinese = chinese
        self.english = english
    @property
    def math(self):
        return self._math
    @math.setter
    def math(self, value):
        if 0 <= value <= 100:
            self._math = value
        else:
            raise ValueError('math score must be between 0 and 100')
    @property
    def chinese(self):
        return self._chinese
    @chinese.setter
    def chinese(self, value):
        if 0 <= value <= 100:
            self._chinese = value
        else:
            raise ValueError('chinese score must be between 0 and 100')
    @property
    def english(self):
        return self._english
    @english.setter
    def english(self, value):
        if 0 <= value <= 100:
            self._english = value
        else:
            raise ValueError('english score must be between 0 and 100')
    def __repr__(self):
        return f"Student: {self.name}, math: {self.math}, chinese: {self.chinese}, english: {self.english}"

stu = Student('Tom', 90, 80, 70)
pp(stu)
print(stu)

Student: Tom, math: 90, chinese: 80, english: 70
Student: Tom, math: 90, chinese: 80, english: 70


In [88]:
@define(slots=False)
class Score:
    def __init__(self, score=0, subject=None) -> None:
        self._score = score
        self._subject = subject
    def __get__(self, instance, owner):
        return self._scocre
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise ValueError(f'{self._subject} score must be int')
        if not 0 <= value <= 100:
            raise ValueError(f'{self._subject} score must be between 0 and 100')
        self._scocre = value
    def __delete__(self):
        del self._scocre

@define(slots=False)
class Student:
    math = Score(score=0, subject='math')
    chinese = Score(score=0, subject='chinese')
    english = Score(score=0, subject='english')
    
    def __attrs_pre_init__(self):
        print('attrs_pre_init')


    """
    attrs: custom init
    
    """ 
    name: str
    math: int
    chinese: int
    english: int

    def __init__(self, name, math, chinese, english):
        self.__attrs_init__(name, math, chinese, english)
        print("custome init")

    """
    attrs: post_init
    """
    # name: str = field()
    # math: int = field()
    # chinese: int = field()
    # english: int = field()

    def __attrs_post_init__(self):
        print('attrs_post_init')
    # def __init__(self, name, math, chinese, english):
    #     self.name = name
    #     self.math = math
    #     self.chinese = chinese
    #     self.english = english

    def __repr__(self):
        return f"Student: {self.name}, math: {self.math}, chinese: {self.chinese}, english: {self.english}"

stu = Student('Tom', 90, 80, 70)
pp(attrs.asdict(stu))
pp(stu)
        

attrs_pre_init
attrs_post_init
custome init
{'chinese': 80, 'english': 70, 'math': 90, 'name': 'Tom'}
Student: Tom, math: 90, chinese: 80, english: 70


In [127]:
@define(slots=False)
class DataDes:
    _score: field(default=0)
    # def __init__(self, score=0):
    #     self._score = score
    def __set__(self, instance, value):
        self._score = value
    def __get__(self, instance, owner):
        print("访问数据描述符里的__get__")
        return self._score

@define(slots=False)
class NonDataDes:
    _score: field(default=0)
    # def __init__(self, score=0):
    #     self._score = score
    def __get__(self, instance, owner):
        print("访问非数据描述符里的__get__")
        return self._score


@define(slots=False)
class Student:
    _math = DataDes(0)
    _chinese = NonDataDes(0)

    _name: str
    _math: int
    _chinese: int
    
    def __getattribute__(self, item):
        print(f"调用__getattribute__{item}")
        return super().__getattribute__(item)
    
    def __repr__(self):
        return f"Student: {self._name}, math: {self._math}, chinese: {self._chinese}"
    
stu = Student(chinese = 90, math = 80, name='Tom')
pp(attrs.asdict(stu))

# pp(attrs.asdict(stu))
# pp(stu._math)
# pp(stu._chinese)
# print(stu)




访问数据描述符里的__get__
访问非数据描述符里的__get__
调用__getattribute____class__
调用__getattribute___name
调用__getattribute___math
调用__getattribute___chinese
{'_chinese': 90, '_math': 80, '_name': 'Tom'}
